In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Acne LDL/code")

In [ ]:
# library
# standard library
import os, sys

# third-party library
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from dataset import dataset_processing
from timeit import default_timer as timer
from utils.report import report_precision_se_sp_yi, report_mae_mse
from utils.utils import Logger, AverageMeter, time_to_str, weights_init
from utils.genLD import genLD
from model.resnet50 import resnet50
import torch.backends.cudnn as cudnn
from transforms.affine_transforms import *
import time
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from timeit import default_timer as timer
import matplotlib.pyplot as plt

from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score,
    f1_score
)

# ============================================================
# LOGGER
# ============================================================
LOG_FILE_NAME = '/content/drive/MyDrive/Acne LDL/code/log_' + time.strftime(
    "%Y-%m-%d_%H-%M-%S", time.localtime()) + '.log'

log = Logger()
log.open(LOG_FILE_NAME, mode="a")

BATCH_SIZE = 32
BATCH_SIZE_TEST = 20
LR = 0.001              # learning rate
NUM_WORKERS = 12
NUM_CLASSES = 4
lr_steps = [30, 60, 90, 120]

# ============================================================
# DEVICE
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ============================================================
# SAVE PATH
# ============================================================
SAVE_MODEL_TPL = "/content/drive/MyDrive/Acne LDL/code/LDL_Fold{idx}.pth"
CM_PNG_TPL     = "/content/drive/MyDrive/Acne LDL/code/LDL_Fold{idx}_confusion.png"

# ============================================================
# TRAIN / VAL / TEST
# ============================================================
def trainval_test(cross_val_index, sigma, lam):

    TRAIN_FILE = f"/content/drive/MyDrive/Acne LDL/code/Classification/NNEW_trainval_{cross_val_index}.txt"
    TEST_FILE  = f"/content/drive/MyDrive/Acne LDL/code/Classification/NNEW_test_{cross_val_index}.txt"
    DATA_PATH  = "/content/drive/MyDrive/Acne LDL/code/Classification/JPEGImages"

    normalize = transforms.Normalize(
        mean=[0.45815152, 0.361242, 0.29348266],
        std=[0.2814769, 0.226306, 0.20132513]
    )

    dset_train = dataset_processing.DatasetProcessing(
        DATA_PATH, TRAIN_FILE,
        transform=transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            RandomRotate(rotation_range=20),
            normalize,
        ])
    )

    dset_test = dataset_processing.DatasetProcessing(
        DATA_PATH, TEST_FILE,
        transform=transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize,
        ])
    )

    train_loader = DataLoader(
        dset_train,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=False
    )

    test_loader = DataLoader(
        dset_test,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=False
    )

    cnn = resnet50().to(device)

    params = [{'params': v, 'lr': LR, 'weight_decay': 5e-4}
              for _, v in cnn.named_parameters() if v.requires_grad]

    optimizer = torch.optim.SGD(params, momentum=0.9)

    loss_func = nn.CrossEntropyLoss().to(device)
    kl_loss_1 = nn.KLDivLoss().to(device)
    kl_loss_2 = nn.KLDivLoss().to(device)
    kl_loss_3 = nn.KLDivLoss().to(device)

    history = {
        "test_acc": [],
        "mae": [],
        "mse": []
    }

    start = timer()

    # ========================================================
    # TRAINING LOOP
    # ========================================================
    for epoch in range(lr_steps[-1]):

        cnn.train()
        losses = AverageMeter()

        for b_x, b_y, b_l in train_loader:

            b_x = b_x.to(device)
            b_l = b_l.numpy() - 1

            ld = genLD(b_l, sigma, 'klloss', 65)
            ld_4 = np.vstack((
                np.sum(ld[:, :5], 1),
                np.sum(ld[:, 5:20], 1),
                np.sum(ld[:, 20:50], 1),
                np.sum(ld[:, 50:], 1)
            )).transpose()

            ld = torch.from_numpy(ld).float().to(device)
            ld_4 = torch.from_numpy(ld_4).float().to(device)

            cls, cou, cou2cls = cnn(b_x, None)

            loss_cls = kl_loss_1(torch.log(cls), ld_4) * 4.0
            loss_cou = kl_loss_2(torch.log(cou), ld) * 65.0
            loss_cls_cou = kl_loss_3(torch.log(cou2cls), ld_4) * 4.0

            loss = (loss_cls + loss_cls_cou) * 0.5 * lam + loss_cou * (1.0 - lam)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses.update(loss.item(), b_x.size(0))

        # ====================================================
        # VALIDATION (START EPOCH >= 9)
        # ====================================================
        if epoch >= 9:
            cnn.eval()

            y_true, y_pred = [], []
            l_true, l_pred = [], []

            with torch.no_grad():
                for x, y, l in test_loader:
                    x = x.to(device)
                    y = y.to(device)

                    cls, cou, cou2cls = cnn(x, None)
                    _, preds = torch.max(cls + cou2cls, 1)

                    y_true.extend(y.cpu().numpy())
                    y_pred.extend(preds.cpu().numpy())

                    _, preds_l = torch.max(cou, 1)
                    preds_l = (preds_l + 1).cpu().numpy()
                    l_true.extend(l.numpy())
                    l_pred.extend(preds_l)

            y_true = np.array(y_true)
            y_pred = np.array(y_pred)
            l_true = np.array(l_true)
            l_pred = np.array(l_pred)

            acc = 100.0 * (y_true == y_pred).mean()
            prec = 100.0 * precision_score(y_true, y_pred, average="macro", zero_division=0)
            rec  = 100.0 * recall_score(y_true, y_pred, average="macro", zero_division=0)
            f1   = 100.0 * f1_score(y_true, y_pred, average="macro", zero_division=0)

            _, MAE, MSE, _ = report_mae_mse(l_true, l_pred, y_true)

            history["test_acc"].append(acc)
            history["mae"].append(MAE)
            history["mse"].append(MSE)

            # ================= CORAL-STYLE PRINT =================
            print(f"Fold {cross_val_index} - Epoch {epoch+1}/{lr_steps[-1]}")
            print("------------------------------------------------------------")
            print(f"Train Loss: {losses.avg:.4f}")
            print(f"Val Acc:   {acc:.2f}%\n")

    # ========================================================
    # CONFUSION MATRIX (PER FOLD)
    # ========================================================
    cm = confusion_matrix(y_true, y_pred)
    fig = plt.figure(figsize=(5, 4))
    ConfusionMatrixDisplay(cm).plot(cmap="Blues")
    plt.title(f"Confusion Matrix Fold {cross_val_index}")
    plt.savefig(CM_PNG_TPL.format(idx=cross_val_index))
    plt.close()

    # ========================================================
    # SAVE MODEL (PER FOLD)
    # ========================================================
    torch.save(cnn.state_dict(), SAVE_MODEL_TPL.format(idx=cross_val_index))

    # ========================================================
    # FINAL PRINT (CORAL STYLE)
    # ========================================================
    print(f"\nTest Set Evaluation Results (Fold {cross_val_index}):")
    print(f"  • Accuracy:  {acc:.2f}%")
    print(f"  • Precision: {prec:.2f}%")
    print(f"  • Recall:    {rec:.2f}%")
    print(f"  • F1-Score:  {f1:.2f}%")
    print(f"  • MAE:       {MAE:.3f}")
    print(f"  • MSE:       {MSE:.3f}\n")

    print(f"Fold {cross_val_index} Summary:")
    print(f"  • Test Accuracy: {acc:.2f}%")
    print(f"  • Test F1-Score: {f1:.2f}%\n")

    return acc, prec, rec, f1, MAE, MSE


# ============================================================
# MAIN (5-FOLD CV)
# ============================================================
cross_val_lists = ['0', '1', '2', '3', '4']
results = []

for fold in cross_val_lists:
    log.write('\n\ncross_val_index: ' + fold + '\n\n')
    results.append(trainval_test(fold, sigma=3.0, lam=0.6))

results = np.array(results)

# ============================================================
# K-FOLD SUMMARY (CORAL STYLE)
# ============================================================
print("\n============================================================")
print("K-FOLD TEST SET EVALUATION RESULTS")
print("============================================================\n")

print(" fold  acc     prec    recall  f1     mae     mse")
for i, r in enumerate(results):
    print(f"{i+1:5d}  {r[0]:6.2f}  {r[1]:6.2f}  {r[2]:6.2f}  {r[3]:6.2f}  {r[4]:6.3f}  {r[5]:6.3f}")

print("\n============================================================")
print("SUMMARY STATISTICS")
print("============================================================\n")

print("📈 Final Test Set Performance :")
print(f"  • Mean Accuracy:  {results[:,0].mean():.2f}% ± {results[:,0].std():.2f}%")
print(f"  • Mean Precision: {results[:,1].mean():.2f}% ± {results[:,1].std():.2f}%")
print(f"  • Mean Recall:    {results[:,2].mean():.2f}% ± {results[:,2].std():.2f}%")
print(f"  • Mean F1-Score:  {results[:,3].mean():.2f}% ± {results[:,3].std():.2f}%")
print(f"  • Mean MAE:       {results[:,4].mean():.3f}")
print(f"  • Mean MSE:       {results[:,5].mean():.3f}")


Device: cuda


cross_val_index: 0

Fold 0 - Epoch 10/120
------------------------------------------------------------
Train Loss: 0.3425
Val Acc:   75.00%

Fold 0 - Epoch 11/120
------------------------------------------------------------
Train Loss: 0.3657
Val Acc:   75.68%

Fold 0 - Epoch 12/120
------------------------------------------------------------
Train Loss: 0.3024
Val Acc:   79.79%

Fold 0 - Epoch 13/120
------------------------------------------------------------
Train Loss: 0.2702
Val Acc:   78.08%

Fold 0 - Epoch 14/120
------------------------------------------------------------
Train Loss: 0.2812
Val Acc:   73.29%

Fold 0 - Epoch 15/120
------------------------------------------------------------
Train Loss: 0.2830
Val Acc:   81.16%

Fold 0 - Epoch 16/120
------------------------------------------------------------
Train Loss: 0.2408
Val Acc:   79.79%

Fold 0 - Epoch 17/120
------------------------------------------------------------
Train Loss: 0.2722
Val Acc:   78.42

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>